In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# #@title Select YOLOv8 🚀 logger {run: 'auto'}
# logger = 'Comet' #@param ['Comet', 'TensorBoard']

# if logger == 'Comet':
#   %pip install -q comet_ml
#   import comet_ml; comet_ml.init()
# elif logger == 'TensorBoard':
#   %load_ext tensorboard
#   %tensorboard --logdir .

###Yolov5

In [ ]:
!pip install ultralytics

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

%cd /content/yolov5/
!pip install -r requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


### 객체 탐지를 수행할 비디오 파일

좌표 추출 & video inference

In [ ]:
# # video inference
# %%time
# !python /content/yolov5/detect.py --weights '/content/drive/MyDrive/best1.pt' --img 1280 --conf 0.4 --source '/content/drive/MyDrive/jytest.mp4'

영상 yolov5(best1.pt)로 돌려서 garbage detection

- 탐지된 결과가 비디오 파일로 /content/yolov5/runs/detect/ 폴더 아래에 저장

In [ ]:
# video inference /content/yolov5/detect.py
# /content/drive/MyDrive/best1.pt

# 객체 탐지를 수행할 비디오 파일 : modelingjun.mp4
# 탐지된 객체가 포함된 비디오의 결과 : /content/yolov5/runs/detect/exp/modelingjun.mp4)
%%time
!python /content/yolov5/detect.py --weights '/content/drive/MyDrive/best1.pt' --img 1280 --conf 0.4 --source '/content/drive/MyDrive/video_and_annotation/video3.mp4'

detect: weights=['/content/drive/MyDrive/best1.pt'], source=/content/drive/MyDrive/video_and_annotation/video3.mp4, data=data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-356-g2070b303 Python-3.10.12 torch-2.3.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/361) /content/drive/MyDrive/video_and_annotation/video3.mp4: 736x1280 (no detections), 979.5ms
video 1/1 (2/361) /content/drive/MyDrive/video_and_annotation/video3.mp4: 736x1280 (no detections), 1040.5ms
video 1/1 (3/361) /content/drive/MyDrive/video_and_annotation/video3.mp4: 736x1280 (no detections

## 쓰레기 무단 투기 감지 코드


1. yolo v5, yolov8 돌려서 프레임별 txt 뽑기>> #프레임구별, 좌표추출

In [ ]:
#프레임 구별, 좌표추출 - yolov5

import subprocess
import csv
import os
from pathlib import Path
import cv2

def run_yolov5_detection(weights, source, img_size=1280, conf_thres=0.4, project='runs/detect', name='exp'):
    # Run the detect.py script with the given parameters
    command = [
        'python', 'detect.py',
        '--weights', weights,
        '--img', str(img_size),
        '--conf', str(conf_thres),
        '--source', source,
        '--save-txt',  # 플래그 옵션
        '--save-conf',
        '--project', project,
        '--name', name
    ]
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True) # 원본영상 detection
        print(result.stdout)
        print(result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        print(f"stdout: {e.stdout}")
        print(f"stderr: {e.stderr}")

def parse_yolov5_results(output_dir, video_path, output_csv_path):
    # Parse the YOLOv5 detection results from the output directory and extract frames
    labels_dir = Path(output_dir) / 'labels'
    if not labels_dir.exists():
        print(f"No labels directory found at {labels_dir}")
        return

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Open the CSV file for writing
    with open(output_csv_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Image Name', 'Class ID', 'Bounding Box', 'Confidence'])

        for label_file in labels_dir.glob('*.txt'):
            image_name = label_file.stem
            try:
                frame_number = int(image_name.split('_')[-1])  # 파일명에서 마지막 부분을 프레임 번호로 추출
            except ValueError:
                print(f"Skipping file with invalid frame number: {image_name}")
                continue

            with open(label_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                    class_id = int(parts[0])
                    bbox = list(map(float, parts[1:5]))
                    conf = float(parts[5]) if len(parts) > 5 else None
                    bbox_str = ','.join(map(str, bbox))

                    # Write to CSV
                    csvwriter.writerow([image_name, class_id, bbox_str, conf])

                    # Frame extraction
                    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
                    ret, frame = cap.read()
                    if not ret:
                        # print(f"Failed to extract frame {frame_number}")
                        continue

                    # Draw bounding box on the frame
                    x_center, y_center, width, height = bbox
                    img_h, img_w = frame.shape[:2]
                    x1 = int((x_center - width / 2) * img_w)
                    y1 = int((y_center - height / 2) * img_h)
                    x2 = int((x_center + width / 2) * img_w)
                    y2 = int((y_center + height / 2) * img_h)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    # Save the frame with the bounding box
                    output_frame_path = Path(output_dir) / 'frames' / f'{image_name}.jpg'
                    output_frame_path.parent.mkdir(parents=True, exist_ok=True)
                    cv2.imwrite(str(output_frame_path), frame)

    # Release the video capture
    cap.release()
    print(f"Results saved to {output_csv_path}")

if __name__ == "__main__":
    weights_path = '/content/drive/MyDrive/best1.pt'
    source_path = '/content/drive/MyDrive/test_video_sample/modelingjun.mp4'
    output_project = '/content/drive/MyDrive/minju_sample/models'
    output_name = 'exp'
    output_csv_path = '/content/drive/MyDrive/minju_sample/detection_results.csv'

    # Run YOLOv5 detection
    run_yolov5_detection(weights=weights_path, source=source_path, project=output_project, name=output_name)

    # Parse the results and save to CSV
    output_dir = Path(output_project) / (output_name)
    parse_yolov5_results(output_dir, source_path, output_csv_path)



KeyboardInterrupt: 

2. #csv파일로 병합 >> 하나의 csv파일로 txt를 묶어줌 >>combined_labels.csv로 뽑힘

In [ ]:
#csv파일로 병합
import csv
import os
from pathlib import Path

def combine_labels_to_csv(labels_dir, output_csv):
    # CSV 파일 열기
    with open(output_csv, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        # 헤더 추가
        csvwriter.writerow(['Frame', 'Class ID', 'x_center', 'y_center', 'width', 'height', 'Confidence'])

        # labels 디렉토리의 모든 txt 파일을 처리
        for label_file in sorted(Path(labels_dir).glob('*.txt')):
            # 파일 이름에서 프레임 번호 추출
            frame = label_file.stem.split('_')[-1]

            with open(label_file, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:  # 5개 이상의 요소가 있는지 확인
                        class_id = parts[0]
                        x_center = parts[1]
                        y_center = parts[2]
                        width = parts[3]
                        height = parts[4]
                        conf = parts[5] if len(parts) > 5 else ''  # 신뢰도는 옵션

                        # CSV 파일에 작성
                        csvwriter.writerow([frame, class_id, x_center, y_center, width, height, conf])

    print(f"CSV 파일이 {output_csv}로 저장되었습니다.")

if __name__ == "__main__":
    labels_dir = '/content/drive/MyDrive/video_and_annotation/label/label_video4'  # labels 디렉토리 경로 : /content/drive/MyDrive/minju_sample/models/exp/labels
    output_csv = '/content/drive/MyDrive/video_and_annotation/result_video4/models/exp/combined_labels.csv'  # 출력 CSV 파일 경로 (결과물) : /content/drive/MyDrive/minju_sample/models/exp/combined_labels.csv

    combine_labels_to_csv(labels_dir, output_csv)


CSV 파일이 /content/drive/MyDrive/video_and_annotation/result_video3/models/exp/combined_labels.csv로 저장되었습니다.


3. classID기준으로 뽑아내기>> filtered_labels.csv

In [ ]:
#classID = 1(garbage)인것만 판별하기

import csv
from pathlib import Path

def extract_class_1_to_csv(labels_dir, output_csv):
    # CSV 파일 열기
    with open(output_csv, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        # 헤더 추가
        csvwriter.writerow(['Frame', 'Class ID', 'x_center', 'y_center', 'width', 'height', 'Confidence'])

        # labels 디렉토리의 모든 txt 파일을 처리
        for label_file in sorted(Path(labels_dir).glob('*.txt')):
            # 파일 이름에서 프레임 번호 추출
            frame = label_file.stem.split('_')[-1]

            with open(label_file, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5 and parts[0] == '0':  # 클래스 ID가 1 인지 확인(garbage)
                        class_id = parts[0]
                        x_center = parts[1]
                        y_center = parts[2]
                        width = parts[3]
                        height = parts[4]
                        conf = parts[5] if len(parts) > 5 else ''  # 신뢰도는 옵션

                        # CSV 파일에 작성
                        csvwriter.writerow([frame, class_id, x_center, y_center, width, height, conf])

    print(f"Class ID가 1인 항목만 {output_csv}로 저장되었습니다.")

if __name__ == "__main__":
    labels_dir = '/content/drive/MyDrive/video_and_annotation/label/label_video3'  # labels 디렉토리 경로
    output_csv = '/content/drive/MyDrive/video_and_annotation/result_video3/models/exp/filtered_labels.csv'  # 출력 CSV 파일 경로 : '/content/drive/MyDrive/minju_sample/models/exp/filtered_labels.csv'

    extract_class_2_to_csv(labels_dir, output_csv)



Class ID가 1인 항목만 /content/drive/MyDrive/video_and_annotation/result_video3/models/exp/filtered_labels.csv로 저장되었습니다.


In [ ]:
#오름차순 정렬
import pandas as pd

def sort_csv_descending(csv_file_path, output_csv_path):
    # CSV 파일 로드
    df = pd.read_csv(csv_file_path)

    # 프레임 번호를 기준으로 내림차순 정렬
    df_sorted = df.sort_values(by='Frame', ascending=True)

    # 정렬된 데이터프레임을 새로운 CSV 파일로 저장
    df_sorted.to_csv(output_csv_path, index=False)

    print(f"정렬된 파일이 {output_csv_path}로 저장되었습니다.")

if __name__ == "__main__":
    input_csv_path = '/content/drive/MyDrive/video_and_annotation/result_video3/models/exp/filtered_labels.csv'  # 정렬할 CSV 파일 경로
    output_csv_path = '/content/drive/MyDrive/video_and_annotation/result_video3/models/exp/filtered_ascending_labels.csv'  # 정렬된 결과를 저장할 CSV 파일 경로

    sort_csv_descending(input_csv_path, output_csv_path)

정렬된 파일이 /content/drive/MyDrive/video_and_annotation/result_video3/models/exp/filtered_ascending_labels.csv로 저장되었습니다.


4. #쓰레기수 증가할때 1 사람수 증가할때 2 코드에서 쓰레기수가 0~1도 감지하는지 확인

In [ ]:
#쓰레기 수 증가할때:1 /사람수 증가할때 :2
import pandas as pd

def find_frames_with_increasing_class_id(csv_file_path):
    # CSV 파일 로드
    df = pd.read_csv(csv_file_path)

    # 프레임별 Class ID가 1인 항목 개수
    class_1_counts = df[df['Class ID'] == 0].groupby('Frame').size()
    class_1_counts_df = class_1_counts.reset_index(name='Count')
    first_frame = class_1_counts_df['Frame'][0]

    # 삽입할 새로운 행 (DataFrame 형태로 만들어줌)
    new_row = pd.DataFrame({'Frame': [first_frame - 1 ], 'Count': [0]})

    # 새로운 행을 기존 DataFrame의 처음에 삽입
    class_1_counts_df = pd.concat([new_row, class_1_counts_df], ignore_index=True)

    print(class_1_counts_df)

    # 이전 프레임보다 Class ID가 1인 항목 개수가 증가한 프레임
    increasing_frames = class_1_counts_df[class_1_counts_df['Count'].diff() > 0]
    last_frame = class_1_counts_df.index[-1]
    if last_frame not in increasing_frames.index:
        last_frame_series = pd.Series([class_1_counts[last_frame]], index=[last_frame])
        increasing_frames = pd.concat([increasing_frames, last_frame_series])

    return increasing_frames

if __name__ == "__main__":
    csv_file_path = '/content/drive/MyDrive/video_and_annotation/result_video3/models/exp/filtered_ascending_labels.csv'  # CSV 파일 경로
    increasing_frames = find_frames_with_increasing_class_id(csv_file_path)

    print("Class ID가 1인 항목이 늘어나는 프레임:")
    print(increasing_frames)

    # 결과를 CSV로 저장하려면 다음 줄을 추가
    increasing_frames.to_csv('/content/drive/MyDrive/video_and_annotation/result_video3/models/exp/increasing_frames.csv', header=True)


     Frame  Count
0       37      0
1       38      1
2       39      1
3       40      1
4       41      1
..     ...    ...
319    356      4
320    357      4
321    358      4
322    359      4
323    360      4

[324 rows x 2 columns]
Class ID가 1인 항목이 늘어나는 프레임:
     Frame  Count    0
1     38.0    1.0  NaN
23    60.0    2.0  NaN
114  151.0    2.0  NaN
193  230.0    3.0  NaN
249  286.0    4.0  NaN
323    NaN    NaN  4.0


### main 함수

In [ ]:
import pandas as pd

def find_last_changes_from_csv(csv_file_path):
    # CSV 파일 로드
    df = pd.read_csv(csv_file_path)
    # df.columns = ['Frame', 'Value']
    frames_series = pd.Series(df['Count'].values, index=df['Frame'])

    # 마지막 프레임의 값을 찾음
    last_value = frames_series.iloc[-2]

    # 마지막 값 이하의 프레임을 필터링
    filtered_series = frames_series[frames_series <= last_value]

    # 각 값의 마지막 변화를 찾음
    unique_values = filtered_series.unique()
    last_changes = {value: filtered_series[filtered_series == value].index[-1] for value in unique_values}

    return last_changes


if __name__ == "__main__":
    csv_file_path = '/content/drive/MyDrive/video_and_annotation/result_video3/models/exp/increasing_frames.csv'  # CSV 파일 경로

    # 마지막 변화 프레임 찾기
    last_changes = find_last_changes_from_csv(csv_file_path)

    print("마지막 변화 프레임:")
    for value, frame in last_changes.items():
        print(f"{int(value)}번째 쓰레기 투척:{int(frame)}번 프레임")



마지막 변화 프레임:
1번째 쓰레기 투척:38번 프레임
2번째 쓰레기 투척:151번 프레임
3번째 쓰레기 투척:230번 프레임
4번째 쓰레기 투척:286번 프레임


In [ ]:
# 쓰레기 무단 투기 {n번째 : m번째 프레임}
last_changes

{1.0: 38.0, 2.0: 151.0, 3.0: 230.0, 4.0: 286.0}